In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!git clone https://huggingface.co/fnlp/SpeechTokenizer
!pip3 install speechtokenizer



In [ ]:



# !export LC_ALL="en_US.UTF-8"
# !export LD_LIBRARY_PATH="/usr/lib64-nvidia"
# !export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
# !ldconfig /usr/lib64-nvidia

In [ ]:
!pip install huggingface_hub
!pip install torchinfo
!pip install wandb
!pip install pytube
!pip install moviepy
!pip install soundfile

In [ ]:
from speechtokenizer import SpeechTokenizer
import soundfile as sf
import torchaudio
import torch
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

batch_size = 4

config_path = '/content/drive/MyDrive/Colab/SpeechTokenizer/speechtokenizer_hubert_avg/config.json'
ckpt_path = '/content/drive/MyDrive/Colab/SpeechTokenizer/speechtokenizer_hubert_avg/SpeechTokenizer.pt'
speech_tokenizer = SpeechTokenizer.load_from_checkpoint(config_path, ckpt_path).to(device)

In [ ]:
from pytube import Playlist
import re
#dowlnoad music files
playlist_url = "https://www.youtube.com/watch?v=Uyi2AS8rzFw&list=PLt_47Imx98wrucH_zNmGr8pFpCqzZYTad"
output_path= '/content/drive/MyDrive/Colab/musicaudio/'
playlist = Playlist(playlist_url)
for video in playlist.videos:

    try:
      audio_stream = video.streams.get_audio_only()
      audio_stream.download(output_path=output_path, filename=video.title + ".mp4")
    except:
      pass

In [ ]:
# !rm -rf /content/drive/MyDrive/Colab/musicaudio\

In [ ]:
from moviepy.editor import AudioFileClip
import os

def towav(mp4_file, output_dir):
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  audio = AudioFileClip(mp4_file)

  duration = int(audio.duration)

  mp4_output_name = os.path.basename(mp4_file).split('.')[0].replace(' ', '_').replace("#", "num")
  for start in range(0, duration, 10):
      outputpath = os.path.join(output_dir, f'{mp4_output_name}_clip_{start}_{start+10}.wav')
      if os.path.exists(outputpath):
          continue
      end = min(start + 10, duration)
      clip = audio.subclip(start, end)
      clip.write_audiofile(outputpath, verbose=False, logger=None)
      
import os
from tqdm import tqdm

audio_files = [os.path.join('/content/drive/MyDrive/Colab/musicaudio' file) for file in os.listdir('/content/drive/MyDrive/Colab/musicaudio') if file.endswith('.mp4')]
print(audio_files)
for af in tqdm(audio_files):
    try:
      towav(af, '/content/drive/MyDrive/Colab/MusicClips')
    except:
      pass

In [ ]:
# !rm -rf /content/drive/MyDrive/Colab/MusicClips

In [ ]:
# !mkdir testmusicfiles

In [ ]:
# !cp  -r /content/testfiles /content/drive/MyDrive/Colab

In [ ]:
!pip install datasets

In [ ]:
# help(load_dataset)

In [ ]:
from datasets import load_dataset
audio_dataset = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/Colab/MusicClips")["train"]


Loading Dataset


Resolving data files:   0%|          | 0/1108 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
speech_tokenizer.sample_rate=16000
#making sure

In [ ]:
# audio_dataset[1]['audio']['sampling_rate']
# # audio_dataset[0]["audio_array"]

In [ ]:
#interpolation from 44100 to 16000
def set_samplerate(w, sr):
    waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.reshape(1, -1)
    waveform = torchaudio.functional.resample(waveform, sr, 16000)
    return waveform


In [ ]:
# x=audio_dataset[1]
# # b=set_samplerate(
# #             torch.tensor(x["audio"]["array"]), x["audio"]["sampling_rate"]
# #         )
# c= torchaudio.functional.resample(torch.tensor(x["audio"]["array"]), 44100, speech_tokenizer.sample_rate)

In [ ]:
# c.shape

In [ ]:
# torch.tensor(x["audio"]["array"]).shape

In [ ]:
audio_dataset = audio_dataset.map(
    lambda x: {
        "newarr": #x["audio"]["array"],#dodtn work
        set_samplerate(
            torch.tensor(x["audio"]["array"]), x["audio"]["sampling_rate"]
        )
    },
    remove_columns=["audio"],
    # keep_in_memory=True,
    writer_batch_size=15000,
)

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

In [ ]:
# speech_tokenizer.sample_rate=44100

In [ ]:
torch.tensor(audio_dataset[0]["newarr"]).shape[1]

160000

In [ ]:


audio_dataset = audio_dataset.filter(
    lambda x: torch.tensor(x["newarr"]).shape[1] ==  16000*10,
writer_batch_size=15000
    )

Filter:   0%|          | 0/1108 [00:00<?, ? examples/s]

In [ ]:
def tokenize(w):
    with torch.no_grad():#make sure
        codes = speech_tokenizer.encode(w.unsqueeze(0).to(device))
    st = codes[:4, :, :].cpu()
    _, L, T = st.shape
    tt = st.transpose(0, 2)
    return tt.reshape(L, T*4)


In [ ]:
torch.tensor(audio_dataset[0]['newarr']).shape
# waveform=torch.tensor(x["audio_array"])

torch.Size([1, 160000])

In [ ]:
# with torch.no_grad():
#   t1=speech_tokenizer.encode(waveform.unsqueeze(0).to(device))

In [ ]:
tokenize(torch.tensor(audio_dataset[0]["newarr"]))

tensor([[744, 484, 511,  ..., 214, 878,  83]])

In [ ]:
audio_dataset = audio_dataset.map(
    lambda x: {"tokens": tokenize(torch.tensor(x["newarr"]))
    # lambda x: {"tokens": torch.tensor(x["audio_array"]) #tried
               },
    remove_columns=[
        "newarr",
    ],
    writer_batch_size=15000,#parallelize
)



In [ ]:
# !rm /content/drive/MyDrive/Colab/testmusicfiles/*

In [ ]:
audio_dataset1=audio_dataset

In [ ]:
def decode(rt):
  B, L = rt.shape
  T = round(L/4) #
  ut=rt.reshape(T, B, 4).transpose(0, 2)
  return speech_tokenizer.decode(ut)


In [ ]:
def save_to_file(tok, filename):
  outputwav = decode(tok.detach().to(device))
  towav(filename, outputwav)


In [ ]:

def save_w(file, w):
  torchaudio.save(file, w[0].detach().cpu(), 16000)


In [ ]:
audio_dataset

DatasetDict({
    train: Dataset({
        features: ['original_sampling_rate', 'tokens'],
        num_rows: 963
    })
    test: Dataset({
        features: ['original_sampling_rate', 'tokens'],
        num_rows: 108
    })
})

In [ ]:
for idx, t in enumerate(audio_dataset1.select(range(0, 10))):
   save_to_file(torch.tensor(t["tokens"]).to(device), f"/content/drive/MyDrive/Colab/testmusicfiles/{idx}_test.wav")

In [ ]:
!mkdir /content/drive/MyDrive/Colab/testmusicfiles

In [ ]:
ad = audio_dataset.with_format('torch')
ad = ad.train_test_split(0.1)
# Setting up the train and test dataloader
train_dl = torch.utils.data.DataLoader(audio_dataset['train'], batch_size=batch_size, shuffle=True)
test_dl = torch.utils.data.DataLoader(audio_dataset['test'], batch_size=batch_size, shuffle=True)
#conventiosnal



In [ ]:
# !pip3 install causal_conv1d_cuda

In [ ]:
# /content/SpeechTokenizer

In [ ]:
# !pip install causal_conv1d_cuda

ERROR: Could not find a version that satisfies the requirement causal_conv1d_cuda (from versions: none)
ERROR: No matching distribution found for causal_conv1d_cuda


In [ ]:
# !CAUSAL_CONV1D_FORCE_BUILD=TRUE pip install causal_conv1d_cuda

ERROR: Could not find a version that satisfies the requirement causal_conv1d_cuda (from versions: none)
ERROR: No matching distribution found for causal_conv1d_cuda


In [ ]:
# !git clone https://github.com/Dao-AILab/causal-conv1d.git
# %cd causal-conv1d
# !git checkout v1.0.2

In [ ]:
# !rm -rf /content/drive/MyDrive/Colab/causal-conv1d

In [ ]:
# !cp -r /content/causal-conv1d /content/drive/MyDrive/Colab/



In [ ]:
# !rm -rf /content/drive/MyDrive/Colab/causal-conv1d/

In [ ]:
# !source /content/drive/MyDrive/colab_env/bin/activate
!cd /content/drive/MyDrive/Colab/causal-conv1d/
!CAUSAL_CONV1D_FORCE_BUILD=TRUE pip install /content/drive/MyDrive/Colab/causal-conv1d/

In [ ]:
# !source /content/drive/MyDrive/colab_env/bin/activate
!cd /content/drive/MyDrive/Colab/mamba
!MAMBA_FORCE_BUILD=TRUE pip install /content/drive/MyDrive/Colab/mamba/

In [ ]:
from tqdm import tqdm
from mamba_ssm import Mamba
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torchinfo
import torchaudio
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
from torch.nn import  functional as F


#setting all variables
epochs = 50
lr = 0.001
block_size = 2000 #length of data maintained by state at once
device = "cuda" if torch.cuda.is_available() else "cpu"
max_iters = 10000
print_iters = 100
eval_iters = 10
eval_interval = 300
n_embed=384 #each index in a discrete set
n_layers = 1
dropout = 0.2
vocab_size=1024 #total number of unique items
from tqdm import tqdm

In [ ]:


class FeedForwardLayer(nn.Module):
  def __init__(self, n_embed) -> None:
    super().__init__()
    self.ffn = nn.Sequential(
      nn.Linear(n_embed, 4*n_embed),
      nn.ReLU(),
      nn.Linear(4*n_embed, n_embed),
      nn.Dropout(dropout),
    )
  def forward(self, x):
    return self.ffn(x)

class BlockLayer(nn.Module):
  def __init__(self, n_embed) -> None:
    super().__init__( )
    self.sa_head = Mamba(
      d_model=n_embed, 
      d_state=16, 
      d_conv=4,  
      expand=1, 
  ).to("cuda")
    self.ffn = FeedForwardLayer(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)


  def forward(self, x):
    x = x + self.sa_head(self.ln1(x))
    x = x + self.ffn(self.ln2(x))

    return x

class MambaAudioModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
    self.position_embedding_table = nn.Embedding(block_size,n_embed)
    self.lm_head = nn.Linear(n_embed,vocab_size)
    self.ffn = FeedForwardLayer(n_embed)
    self.block = BlockLayer(n_embed)#just one block to start with


  def forward(self, idx, targets=None):
    idx = idx[:,-block_size:]#picking just 1999
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx) # 4 x 1999 x 384
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # 1999 X 384
    x = tok_emb + pos_emb # posittional encoding added to each block
    x = self.block(x) # (B,T,Q, C_e)
    logits = self.lm_head(x) # (B,T,vocab_size)
    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      # loss = F.mse_loss(logits, targets)
      # loss = F.l1_loss(logits, targets)
      loss = F.cross_entropy(logits, targets)
      logits = logits.view(B,T,C)
    return logits, loss







In [ ]:
tok_emb = model.token_embedding_table(x)
tok_emb.shape

torch.Size([4, 1999, 384])

In [ ]:
pos_emb = model.position_embedding_table(torch.arange(T,device=device))
pos_emb.shape

torch.Size([1999, 384])

In [ ]:
x = tok_emb + pos_emb
x.shape

torch.Size([4, 1999, 384])

In [ ]:
bl=Block(n_embed).to(device)

In [ ]:
temp1 = bl.ln1(x)
temp1.shape

torch.Size([4, 1999, 384])

In [ ]:
bl.sa_head.parameters()

<generator object Module.parameters at 0x7dbf7f6fa7a0>

In [ ]:
for name, param in bl.sa_head.named_parameters():
    print(f"Parameter: {name}, Shape: {param.shape}")

Parameter: A_log, Shape: torch.Size([384, 16])
Parameter: D, Shape: torch.Size([384])
Parameter: in_proj.weight, Shape: torch.Size([768, 384])
Parameter: conv1d.weight, Shape: torch.Size([384, 1, 4])
Parameter: conv1d.bias, Shape: torch.Size([384])
Parameter: x_proj.weight, Shape: torch.Size([56, 384])
Parameter: dt_proj.weight, Shape: torch.Size([384, 24])
Parameter: dt_proj.bias, Shape: torch.Size([384])
Parameter: out_proj.weight, Shape: torch.Size([384, 384])


In [ ]:
bl.sa_head(bl.ln1(x))


In [ ]:
def caltesloss(model, dataset):
  model.eval()
  test_losses = []
  with torch.no_grad():
    for tokens in tqdm(dataset['test']['tokens']):#get from earlier splitted dataset 
      tokens = tokens.to(device)
      x = tokens[:,:-1].contiguous()
      y = tokens[:,1:].contiguous()
      logits, loss = model(x,y )
      test_losses.append(loss)
  model.train()
  return sum(test_losses)/len(test_losses)




def outputmsic(filename, model, testdata):
  firstbit = testdata.shape[-1]//2
  tokens = testdata[:firstbit]
  tokens = tokens.testdata(1,firstbit)
  max_new_tokens=testdata.shape[-1]-firstbit
  idx = tokens.to(device)
  idx_next = []
  for i in tqdm(range(max_new_tokens)):
    idx_cond = idx[:,-block_size:]
    logits, loss = model(idx_cond)
    last_timestep = logits[:,-1,:]
    probs = F.softmax(last_timestep, dim=1)
    next_index = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, next_index), dim=1)

  save_to_file(idx, f"{filename}_test.wav")
  save_to_file(testdata.unsqueeze(0), f"{filename}_input.wav")

In [ ]:
# def save_model_checkpoint(ind, model, dataset):
#   model_path = f"/content/drive/MyDrive/Colab/checkpoints/{ind}_model1.pt"
#   torch.save(model.state_dict(), model_path)
#   wandb.save(model_path)
#   train_filename = f"/content/drive/MyDrive/Colab/checkpoints/{ind}_train"
#   test_filename = f"/content/drive/MyDrive/Colab/checkpoints/{ind}_test"
#   outputmsic(train_filename, model, dataset['train'][0]['tokens'][0])
#   outputmsic(test_filename, model, dataset['test'][0]['tokens'][0])

#not needed as of now

In [ ]:
import wandb
import time
import torchinfo
from tqdm.notebook import tqdm
wandb.init(project="CSE576-mamba")
model = MambaAudioModel(vocab_size).to(device)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
# torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
model.train()
ind = 0
start_time = time.time()

print(torchinfo.summary(model))





layers 1
Layer (type:depth-idx)                   Param #
MambaAudioModel                          --
├─Embedding: 1-1                         393,216
├─Embedding: 1-2                         768,000
├─Linear: 1-3                            394,240
├─FeedForward: 1-4                       --
│    └─Sequential: 2-1                   --
│    │    └─Linear: 3-1                  591,360
│    │    └─ReLU: 3-2                    --
│    │    └─Linear: 3-3                  590,208
│    │    └─Dropout: 3-4                 --
├─Sequential: 1-5                        --
│    └─Block: 2-2                        --
│    │    └─Mamba: 3-5                   481,920
│    │    └─FeedForward: 3-6             1,181,568
│    │    └─LayerNorm: 3-7               768
│    │    └─LayerNorm: 3-8               768
Total params: 4,402,048
Trainable params: 4,402,048
Non-trainable params: 0


In [ ]:
!pip install torchviz

In [ ]:
from torchviz import make_dot

make_dot(logits, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

'rnn_torchviz.png'

In [ ]:
i, data = next(enumerate(train_dl, 0))

In [ ]:
batch_size,block_size

(4, 2000)

In [ ]:
data['tokens'].shape

torch.Size([4, 1, 2000])

In [ ]:
tokens = data['tokens'].to(device).reshape(batch_size,2000)

In [ ]:
tokens.shape

torch.Size([4, 2000])

In [ ]:
x= tokens[:,:-1].contiguous()

In [ ]:
tokens.shape

torch.Size([4, 2000])

In [ ]:
x,x.shape

(tensor([[ 702,  397,  133,  ...,   63,  232,  586],
         [ 328,   99,   97,  ...,  178,  232,  802],
         [ 328,  214,  985,  ...,  328,  802,  188],
         [ 328,  857, 1010,  ...,  538,  537,   78]], device='cuda:0'),
 torch.Size([4, 1999]))

In [ ]:
y= tokens[:,1:].contiguous()

In [ ]:
y,y.shape

(tensor([[ 397,  133,  848,  ...,  232,  586,  839],
         [  99,   97,  534,  ...,  232,  802,  271],
         [ 214,  985,  942,  ...,  802,  188,   22],
         [ 857, 1010,   35,  ...,  537,   78,  316]], device='cuda:0'),
 torch.Size([4, 1999]))

In [ ]:
logits, loss = model(torch.tensor(x),torch.tensor(y))

<ipython-input-122-775375e8b9ab>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  logits, loss = model(torch.tensor(x),torch.tensor(y))


In [ ]:
loss

tensor(7.2695, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
logits

tensor([[[-1.2748,  0.7497, -0.8749,  ..., -0.6090,  0.3453,  0.9694],
         [-0.9396,  0.7430,  0.4219,  ..., -1.3958,  0.0399,  0.1222],
         [-0.1528, -0.2888,  0.7732,  ..., -0.3747,  0.1090,  0.2212],
         ...,
         [ 0.9775,  0.4363,  0.7908,  ..., -0.3579,  0.2748,  0.0202],
         [-1.0461,  0.6023,  0.1777,  ..., -0.9906,  0.1746, -1.1180],
         [-0.7334,  0.0631,  1.0477,  ..., -0.9140, -1.6884,  0.0214]],

        [[-2.0731,  0.0125, -0.1541,  ..., -0.7961,  1.1018,  0.1662],
         [-0.4173,  0.5670,  0.4688,  ..., -0.7980, -0.3774,  0.3261],
         [-0.0197,  1.2453, -0.6483,  ..., -0.9032,  0.9482,  0.2851],
         ...,
         [-0.4842,  0.0218,  0.6771,  ..., -0.4239,  0.0918, -0.4925],
         [ 0.7212,  0.5451, -0.4096,  ...,  0.3644,  1.6763, -0.6163],
         [-1.4398, -0.6195, -0.1353,  ..., -0.9860,  0.0517,  0.9339]],

        [[-2.0731,  0.0125, -0.1541,  ..., -0.7961,  1.1018,  0.1662],
         [-1.0680,  0.4273,  0.5556,  ..., -1

In [ ]:
ind

0

In [ ]:
for epoch in tqdm(range(epochs)):
  for batch in tqdm(train_dl):
    if batch['tokens'].shape[-1] != block_size:
      continue
    batch_size = batch['tokens'].shape[0]
    tokens = batch['tokens'].to(device).reshape(batch_size,block_size)

    x = tokens[:,:-1].contiguous()
    y = tokens[:,1:].contiguous()
    logits, loss = model(x,y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # Prevents gradient explosion.
    torch.nn.utils.clip_grad.clip_grad_norm_(model.parameters(), 1.0)#experiment with threshold
    optimizer.step()
    ind += 1



/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py

In [ ]:
# model_path="/content/drive/MyDrive/Colab/checkpoints/overfittedmodel.pt"
# torch.save(model.state_dict(), model_path)
# wandb.save(model_path)
# wandb.finish()
# print(torchinfo.summary(model))

# if reoading is required

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Iteration,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Testing Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training Loss,█▆▅▃▇▅▂▅▇▆▆▃▁▅▆▃▅▆▅▄▅▅▄▂▆▆▆▅▄▂▆▅▄▆▆▃▄▅▆▁
time_elapsed,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
Epoch,49
Iteration,12040
Testing Loss,4.01582
Training Loss,4.23851
time_elapsed,118.75676


Layer (type:depth-idx)                   Param #
MambaAudioModel                          --
├─Embedding: 1-1                         393,216
├─Embedding: 1-2                         768,000
├─Linear: 1-3                            394,240
├─FeedForward: 1-4                       --
│    └─Sequential: 2-1                   --
│    │    └─Linear: 3-1                  591,360
│    │    └─ReLU: 3-2                    --
│    │    └─Linear: 3-3                  590,208
│    │    └─Dropout: 3-4                 --
├─Sequential: 1-5                        --
│    └─Block: 2-2                        --
│    │    └─Mamba: 3-5                   481,920
│    │    └─FeedForward: 3-6             1,181,568
│    │    └─LayerNorm: 3-7               768
│    │    └─LayerNorm: 3-8               768
Total params: 4,402,048
Trainable params: 4,402,048
Non-trainable params: 0


In [ ]:
model.eval()

MambaAudioModel(
  (token_embedding_table): Embedding(1024, 384)
  (position_embedding_table): Embedding(2000, 384)
  (lm_head): Linear(in_features=384, out_features=1024, bias=True)
  (ffn): FeedForward(
    (ffn): Sequential(
      (0): Linear(in_features=384, out_features=1536, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1536, out_features=384, bias=True)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (blocks): Sequential(
    (0): Block(
      (sa_head): Mamba(
        (in_proj): Linear(in_features=384, out_features=768, bias=False)
        (conv1d): Conv1d(384, 384, kernel_size=(4,), stride=(1,), padding=(3,), groups=384)
        (act): SiLU()
        (x_proj): Linear(in_features=384, out_features=56, bias=False)
        (dt_proj): Linear(in_features=24, out_features=384, bias=True)
        (out_proj): Linear(in_features=384, out_features=384, bias=False)
      )
      (ffn): FeedForward(
        (ffn): Sequential(
          (0): Linear(in_features=384, out_fe

In [ ]:
i, data = next(enumerate(train_dl, 0))
batch_size = batch['tokens'].shape[0]
tokens = batch['tokens'].to(device).reshape(batch_size,block_size)
xt = tokens[:,:-1].contiguous()
yt=model(xt)

In [ ]:
towav('/content/drive/MyDrive/Colab/checkpoints/result', model, yt)

In [ ]:
idx = torch.tensor([[10,]]).to(device)

  0%|          | 0/1999 [00:00<?, ?it/s]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
vocab_size = 16000  # Vocabulary size
d_model = 512  # Dimensionality of the model
num_heads = 8  # Number of attention heads
dff = 2048  # Dimensionality of the feed-forward layer
num_layers = 6  # Number of transformer layers
max_len = 16000  # Maximum sequence length
block_size =16000
epochs=50

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)#standard
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)#not a parameter

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class TransformerNextTokenGenerator(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, dff, num_layers, max_len=5000):
        super(TransformerNextTokenGenerator, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        self.transformer_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=dff)
        self.transformer = nn.TransformerEncoder(self.transformer_layers, num_layers)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, input_sequence):
        input_embedded = self.embedding(input_sequence) * np.sqrt(self.d_model)
        input_with_position = self.positional_encoding(input_embedded)
        transformer_output = self.transformer(input_with_position)
        next_token_logits = self.fc(transformer_output[-1, :])  # Only use the last token's output
        return next_token_logits




In [ ]:
# wandb.init(project="transformer test")

In [ ]:

modelt = TransformerNextTokenGenerator(vocab_size, d_model, num_heads, dff, num_layers, max_len=max_len)
modelt.to(device)
input_sequence = torch.randint(0, vocab_size, (vocab_size,)).to(device)
# input_sequence = torch.randint(0, vocab_size, (max_len,))

# Generate next token logits
next_token_logits = modelt(input_sequence.unsqueeze(1))

next_token_logits.shape


Next token logits shape: torch.Size([1, 16000])


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
for epoch in tqdm(range(epochs)):
  for batch in tqdm(train_dataloader):
    if batch['tokens'].shape[-1] != block_size:
      continue
    batch_size = batch['tokens'].shape[0]
    tokens = batch['tokens'].to(device).reshape(batch_size,block_size)

    x = tokens[:,0:3200].contiguous()
    y = tokens[:,1:3201].contiguous()
    print(x.shape)
    logits = modelt(x)
    loss = loss_fn(logits.view(-1, vocab_size), y[0].view(-1))
    if ind % 10 == 0:
      print(loss)
      wandb.log({"Training Loss":loss.item(), "Iteration":ind, "Epoch":epoch, "time_elapsed":time.time()-start_time})
    if ind % 500 == 0:
      tl = caltesloss(model, audio_dataset)
      print("testloaximum possible lossss", tl)
      wandb.log({"Testing Loss":tl.item(), "Iteration":ind, "Epoch":epoch})
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    ind += 1

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

...

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

In [ ]:
del input_sequence
del model


In [ ]:
clear_cuda()

In [ ]:
input_sequence = torch.randint(0, vocab_size, (max_len,))

In [ ]:
def clear_cuda():
    # Clear memory associated with tensors
    torch.cuda.empty_cache()

    # Deallocate memory associated with models
    for obj in list(globals().items()):
        _, obj = obj
        if torch.is_tensor(obj):
            del obj
    torch.cuda.empty_cache()

In [ ]:
clear_cuda()

In [ ]:
del input_sequence
del model

In [ ]:
import causal_conv1d

print(causal_conv1d.__version__)

In [ ]:
!python3 -c 'import torch; print(torch.version.cuda)'
!nvcc --version

12.1
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
# !git clone https://github.com/Dao-AILab/causal-conv1d.git
# !cd causal_conv1d
# !git checkout v1.0.2  # this is the highest compatible version allowed by Mamba
# !CAUSAL_CONV1D_FORCE_BUILD=TRUE pip install .

Cloning into 'causal-conv1d'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 161 (delta 36), reused 41 (delta 20), pack-reused 93
Receiving objects: 100% (161/161), 60.47 KiB | 1.21 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/bin/bash: line 1: cd: causal_conv1d: No such file or directory
fatal: not a git repository (or any of the parent directories): .git
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
# !update-alternatives --config cuda

There is only one alternative in link group cuda (providing /usr/local/cuda): /usr/local/cuda-12.2
Nothing to configure.


In [ ]:
!pip install causal-conv1d